# Improving Employee Retention by Predicting Employee Attrition Using Machine Learning

## Import Library and Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_raw = pd.read_csv(r'Improving Employee Retention by Predicting Employee Attrition Using Machine Learning.xlsx - hr_data.csv')
pd.set_option('display.max_column', None)
data_raw.head()

,Username,EnterpriseID,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,NomorHP,Email,TingkatPendidikan,PernahBekerja,IkutProgramLOP,AlasanResign,TanggalLahir,TanggalHiring,TanggalPenilaianKaryawan,TanggalResign
0,spiritedPorpoise3,111065,Belum_menikah,Pria,Outsource,Software Engineer (Back End),Freshgraduate_program,Sangat_bagus,Jakarta Timur,Employee_Referral,4,4.0,0.0,0.0,9.0,+6282232522xxx,spiritedPorpoise3135@yahoo.com,Magister,1,1.0,masih_bekerja,1972-07-01,2011-01-10,2016-2-15,-
1,jealousGelding2,106080,Belum_menikah,Pria,FullTime,Data Analyst,Freshgraduate_program,Sangat_kurang,Jakarta Utara,Website,4,4.0,4.0,0.0,3.0,+6281270745xxx,jealousGelding2239@yahoo.com,Sarjana,1,1.0,toxic_culture,1984-04-26,2014-01-06,2020-1-17,2018-6-16
2,pluckyMuesli3,106452,Menikah,Pria,FullTime,Software Engineer (Front End),Freshgraduate_program,Bagus,Jakarta Timur,Indeed,4,3.0,0.0,0.0,11.0,+6281346215xxx,pluckyMuesli3961@icloud.com,Magister,1,1.0,jam_kerja,1974-01-07,2011-01-10,2016-01-10,2014-9-24
3,stressedTruffle1,106325,Belum_menikah,Pria,Outsource,Software Engineer (Front End),Freshgraduate_program,Bagus,Jakarta Pusat,LinkedIn,3,3.0,0.0,4.0,6.0,+6283233846xxx,stressedTruffle1406@hotmail.com,Sarjana,1,0.0,masih_bekerja,1979-11-24,2014-2-17,2020-02-04,-
4,shyTermite7,111171,Belum_menikah,Wanita,FullTime,Product Manager,Freshgraduate_program,Bagus,Jakarta Timur,LinkedIn,3,3.0,0.0,0.0,11.0,+6287883263xxx,shyTermite7149@gmail.com,Sarjana,1,0.0,ganti_karir,1974-11-07,2013-11-11,2020-1-22,2018-09-06


## Data Understanding

In [3]:
data_raw.shape

(287, 25)

In [4]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 25 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Username                            287 non-null    object 
 1   EnterpriseID                        287 non-null    int64  
 2   StatusPernikahan                    287 non-null    object 
 3   JenisKelamin                        287 non-null    object 
 4   StatusKepegawaian                   287 non-null    object 
 5   Pekerjaan                           287 non-null    object 
 6   JenjangKarir                        287 non-null    object 
 7   PerformancePegawai                  287 non-null    object 
 8   AsalDaerah                          287 non-null    object 
 9   HiringPlatform                      287 non-null    object 
 10  SkorSurveyEngagement                287 non-null    int64  
 11  SkorKepuasanPegawai                 282 non-n

In [5]:
sum_missing = data_raw.isna().sum()
percent_missing = round(data_raw.isnull().sum() *100 / len(data_raw),2)
missing_value_df = pd.DataFrame({'feature_name': data_raw.columns,
                                 'sum_missing': sum_missing,
                                 'percent_missing': percent_missing}).reset_index(drop=True).sort_values(by='sum_missing', ascending=False)
pd.set_option('display.max_rows', None)
missing_value_df.head(100)

,feature_name,sum_missing,percent_missing
19,IkutProgramLOP,258,89.90
20,AlasanResign,66,23.00
14,JumlahKetidakhadiran,6,2.09
11,SkorKepuasanPegawai,5,1.74
12,JumlahKeikutsertaanProjek,3,1.05
13,JumlahKeterlambatanSebulanTerakhir,1,0.35
23,TanggalPenilaianKaryawan,0,0.00
22,TanggalHiring,0,0.00
21,TanggalLahir,0,0.00
18,PernahBekerja,0,0.00


In [6]:
data_raw.describe().T

,count,mean,std,min,25%,50%,75%,max
EnterpriseID,287.0,105923.324042,4044.977599,100282.0,101269.0,106069.0,110514.5,111703.0
SkorSurveyEngagement,287.0,3.101045,0.836388,1.0,3.0,3.0,4.0,5.0
SkorKepuasanPegawai,282.0,3.904255,0.913355,1.0,3.0,4.0,5.0,5.0
JumlahKeikutsertaanProjek,284.0,1.179577,2.294441,0.0,0.0,0.0,0.0,8.0
JumlahKeterlambatanSebulanTerakhir,286.0,0.412587,1.275016,0.0,0.0,0.0,0.0,6.0
JumlahKetidakhadiran,281.0,10.448399,6.902252,1.0,5.0,10.0,15.0,55.0
IkutProgramLOP,29.0,0.517241,0.508548,0.0,0.0,1.0,1.0,1.0


In [7]:
data_raw.select_dtypes(include='object').describe().T

,count,unique,top,freq
Username,287,285,boredEggs0,2
StatusPernikahan,287,5,Belum_menikah,132
JenisKelamin,287,2,Wanita,167
StatusKepegawaian,287,3,FullTime,217
Pekerjaan,287,14,Software Engineer (Back End),109
JenjangKarir,287,3,Freshgraduate_program,169
PerformancePegawai,287,5,Biasa,85
AsalDaerah,287,5,Jakarta Pusat,72
HiringPlatform,287,9,Indeed,85
NomorHP,287,287,+6282232522xxx,1


In [8]:
data_raw.nunique()

Username                              285
EnterpriseID                          287
StatusPernikahan                        5
JenisKelamin                            2
StatusKepegawaian                       3
Pekerjaan                              14
JenjangKarir                            3
PerformancePegawai                      5
AsalDaerah                              5
HiringPlatform                          9
SkorSurveyEngagement                    5
SkorKepuasanPegawai                     5
JumlahKeikutsertaanProjek               9
JumlahKeterlambatanSebulanTerakhir      7
JumlahKetidakhadiran                   22
NomorHP                               287
Email                                 287
TingkatPendidikan                       3
PernahBekerja                           2
IkutProgramLOP                          2
AlasanResign                           11
TanggalLahir                          284
TanggalHiring                          97
TanggalPenilaianKaryawan          

## Preprocessing

### Feature Engineering

Sebelum melakukan preprocessing data lainnya, ada baiknya kita drop beberapa feature yang nantinya tidak/kurang berguna untuk analisis maupun untuk modeling:
1. Feature identitas seperti `Username, EnterpriseID, NomorHP,Email`
2. `IkutProgramLOP` karena mengandung terlalu banyak missing value
3. `TanggalPenilaianKaryawan` karena banyak ditemukan data yang kurang valid(tanggal penilaian setelah karyawan resign)

In [9]:
feature_drop = [# Identitas
                'Username',
                'EnterpriseID',
                'NomorHP',
                'Email',
                # Missing value > 80%
                'IkutProgramLOP',
                # Invalid
                'TanggalPenilaianKaryawan'
]

In [10]:
data_pre = data_raw.drop(feature_drop, axis=1)

In [11]:
data_pre.head(1)

,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,TingkatPendidikan,PernahBekerja,AlasanResign,TanggalLahir,TanggalHiring,TanggalResign
0,Belum_menikah,Pria,Outsource,Software Engineer (Back End),Freshgraduate_program,Sangat_bagus,Jakarta Timur,Employee_Referral,4,4.0,0.0,0.0,9.0,Magister,1,masih_bekerja,1972-07-01,2011-01-10,-


#### Labeling/Define Target

Dalam modeling ini, tujuan utamanya untuk memprediksi pengunduran diri karyawan berdasarkan dari perilaku, karakteristik, dan faktor lain dari feature yang tersedia. Oleh karena itu, target dari dataset ini menjelaskan apakah karyawan tersebut resign atau tidak
<br>
<br>
Dalam dataset ini feature `AlasanResign` dapat digunakan untuk menjelaskan apakah karyawan tersebut resign atau tidak

In [12]:
data_pre['AlasanResign'].value_counts()

masih_bekerja               132
jam_kerja                    16
ganti_karir                  14
kejelasan_karir              11
tidak_bisa_remote            11
toxic_culture                10
leadership                    9
tidak_bahagia                 8
internal_conflict             4
Product Design (UI & UX)      4
apresiasi                     2
Name: AlasanResign, dtype: int64

Feature `AlasanResign` memiliki beberapa value yang menjelaskan apakah karyawan tersebut `masih_bekerja` atau tidak, dan masing-masing alasan seorang karyawan memilih untuk resign
<br> Jadi untuk labeling kita akan membuat feature `resign` dengan value 0 (masih bekerja) dan 1(resign).
<br> Namun ada satu yang unik disini dimana ada `AlasanResign` dengan value `Product Design (UI & UX)` dimana hal tersebut adalah posisi dari karyawan tersebut.

In [13]:
data_pre[(data_pre['AlasanResign']=='Product Design (UI & UX)')].head()

,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,TingkatPendidikan,PernahBekerja,AlasanResign,TanggalLahir,TanggalHiring,TanggalResign
27,Menikah,Wanita,FullTime,Software Engineer (Front End),Senior_level,Biasa,Jakarta Selatan,LinkedIn,3,5.0,0.0,0.0,15.0,Sarjana,1,Product Design (UI & UX),1987-04-10,2016-07-06,2016-08-07
124,Belum_menikah,Pria,FullTime,Product Design (UI & UX),Freshgraduate_program,Sangat_bagus,Jakarta Utara,Diversity_Job_Fair,3,3.0,0.0,0.0,19.0,Sarjana,1,Product Design (UI & UX),1989-05-02,2011-9-26,2017-12-15
147,Menikah,Wanita,FullTime,Software Engineer (Front End),Freshgraduate_program,Sangat_bagus,Jakarta Timur,LinkedIn,3,5.0,0.0,0.0,20.0,Sarjana,1,Product Design (UI & UX),1969-02-09,2012-03-07,2016-4-24
203,Menikah,Wanita,FullTime,Software Engineer (Back End),Mid_level,Sangat_bagus,Jakarta Selatan,Website,3,5.0,0.0,0.0,9.0,Sarjana,1,Product Design (UI & UX),1988-08-29,2012-01-09,2018-04-01


Setelah diteliti lebih lanjut, karyawan dengan `AlasanResign` Product Design (UI & UX) sudah ada tanggal resignnya, jadi hal tersebut kemungkinan adalah kesalahan pada proses input data

In [14]:
data_pre['AlasanResign'].isna().sum()

66

Terdapat 66 baris yang kosong dalam feature `AlasanResign` untuk memastikan akan diisi apa mari kita apakah karyawan tersebut sudah resign atau belum

In [15]:
data_res = data_pre[(data_pre['AlasanResign'].isnull()) & (data_pre['TanggalResign']=='-')]
data_res.shape

(66, 19)

Hal tersebut membuktikan bahwa pada feature `AlasanResign` baris yang kosong merupakan karyawan yang masih bekerja semua, oleh karena itu kita akan imputasi masih_bekerja pada baris yang kosong

In [16]:
data_pre['AlasanResign'].fillna('masih_bekerja', inplace=True)

In [17]:
data_pre['resign'] = np.where((data_pre['AlasanResign']=='masih_bekerja'), 0, 1)

In [18]:
data_pre['resign'].value_counts(normalize=True)*100

0    68.989547
1    31.010453
Name: resign, dtype: float64

### Feature Modification

#### TanggalLahir

Dengan feature tanggal lahir kita akan memodifikasinya menjadi feature umur. Jika dilihat dari feature tanggal lainnya, dataset ini memuat informasi karyawan sampai dengan tahun 2020, dengan asumsi bahwa dataset ini digunakan pada awal tahun 2021, maka dengan mengurangi tanggal 2021-01-01 dengan `TanggalLahir` kita bisa mengetahui umur dari masing-masing karyawan

In [19]:
data_pre['TanggalLahir'].head(2)

0    1972-07-01
1    1984-04-26
Name: TanggalLahir, dtype: object

In [20]:
data_pre['TanggalLahir'] = pd.to_datetime(data_pre['TanggalLahir'], format='%Y-%m-%d')
data_pre['umur'] = round(pd.to_numeric((pd.to_datetime('2020-12-01') - data_pre['TanggalLahir']) / np.timedelta64(1, 'Y')))

In [21]:
data_pre['umur'].describe()

count    287.000000
mean      41.804878
std        8.880928
min       28.000000
25%       34.500000
50%       40.000000
75%       47.000000
max       70.000000
Name: umur, dtype: float64

#### TanggalHiring dan TanggalResign

Dari kedua feature ini kita dapat mengetahui berapa lama waktu dari seorang karyawan bekerja dalam bulan dengan cara `TanggalResign` dikurangi `TanggalHiring`. Untuk karyawan yang masih bekerja, akan dikurangi dengan tanggal 2021-01-01

In [22]:
# mengisi TanggalResign dengan 2020-12-01 untuk karyawan yang belum resign
data_pre['TanggalResign'] = data_pre['TanggalResign'].where(data_pre['TanggalResign'] != '-', '2021-01-01')

In [23]:
data_pre['TanggalHiring'] = pd.to_datetime(data_pre['TanggalHiring'],format='%Y-%m-%d')
data_pre['TanggalResign'] = pd.to_datetime(data_pre['TanggalResign'],format='%Y-%m-%d')

In [24]:
# hitung perbedaan tanggal
data_pre['LamaBekerja'] = data_pre['TanggalResign'] - data_pre['TanggalHiring']
# konversi perbedaan tanggal ke tahun
data_pre['LamaBekerja'] = round(data_pre['LamaBekerja'].apply(lambda x: x.days/365))

In [25]:
data_pre['LamaBekerja'].describe()

count    287.000000
mean       6.749129
std        2.867674
min       -4.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       15.000000
Name: LamaBekerja, dtype: float64

In [26]:
data_pre[data_pre['LamaBekerja']<0].head(3)

,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,TingkatPendidikan,PernahBekerja,AlasanResign,TanggalLahir,TanggalHiring,TanggalResign,resign,umur,LamaBekerja
108,Belum_menikah,Wanita,FullTime,Data Analyst,Freshgraduate_program,Biasa,Jakarta Timur,CareerBuilder,4,4.0,0.0,0.0,10.0,Sarjana,1,toxic_culture,1976-09-22,2015-03-30,2014-08-19,1,44.0,-1.0
207,Menikah,Wanita,FullTime,Software Engineer (Front End),Mid_level,Sangat_bagus,Jakarta Pusat,Indeed,4,3.0,5.0,0.0,17.0,Sarjana,1,leadership,1972-11-21,2017-01-07,2013-05-30,1,48.0,-4.0


Setelah dilakukan proses diatas, terdapat 2 karyawan yang `LamaBekerja`nya bervalue minus, setelah diselidiki hal tersebut karena TanggalResign lebih awal dari TanggalHiring. Asumsi terkuatnya adalah kesalahan input sehingga kedua tanggal tersebut terbalik, namun untuk saat ini kita akan drop saja kedua karyawan tersebut

In [27]:
data_pre.drop(data_pre[data_pre['LamaBekerja']<0].index, inplace=True)

### Missing Value

In [28]:
sum_missing = data_pre.isna().sum()
percent_missing = round(data_pre.isnull().sum() *100 / len(data_pre),2)
missing_value = pd.DataFrame({'feature_name': data_pre.columns,
                                 'sum_missing': sum_missing,
                                 'percent_missing': percent_missing}).reset_index(drop=True).sort_values(by='sum_missing', ascending=False)
pd.set_option('display.max_rows', None)
missing_value.head(4)

,feature_name,sum_missing,percent_missing
12,JumlahKetidakhadiran,6,2.11
9,SkorKepuasanPegawai,5,1.75
10,JumlahKeikutsertaanProjek,3,1.05
11,JumlahKeterlambatanSebulanTerakhir,1,0.35


In [29]:
data_pre['JumlahKetidakhadiran'].fillna(0, inplace=True)
data_pre['SkorKepuasanPegawai'].fillna(0, inplace=True)
data_pre['JumlahKeikutsertaanProjek'].fillna(0, inplace=True)
data_pre['JumlahKeterlambatanSebulanTerakhir'].fillna(0, inplace=True)

### Feature Selection

In [30]:
hm = data_pre.corr()
fig = px.imshow(hm, text_auto=".2f", color_continuous_scale='PuRd',
                template='plotly_dark',width=900,height=900)
fig.show()

#### Checking Nunique

In [31]:
data_pre.select_dtypes(include='object').nunique()

StatusPernikahan       5
JenisKelamin           2
StatusKepegawaian      3
Pekerjaan             14
JenjangKarir           3
PerformancePegawai     5
AsalDaerah             5
HiringPlatform         9
TingkatPendidikan      3
PernahBekerja          2
AlasanResign          11
dtype: int64

In [32]:
objects = data_pre.dtypes[data_pre.dtypes == "object"].index

In [33]:
for col in objects:
    print(f'''Value count feature {col}:''')
    print(data_pre[col].value_counts())
    print()

Value count feature StatusPernikahan:
Belum_menikah    131
Menikah           56
Lainnya           48
Bercerai          47
-                  3
Name: StatusPernikahan, dtype: int64

Value count feature JenisKelamin:
Wanita    165
Pria      120
Name: JenisKelamin, dtype: int64

Value count feature StatusKepegawaian:
FullTime      215
Outsource      66
Internship      4
Name: StatusKepegawaian, dtype: int64

Value count feature Pekerjaan:
Software Engineer (Back End)      109
Software Engineer (Front End)      71
Software Engineer (Android)        24
Product Design (UI & UX)           24
Product Manager                    17
Data Analyst                       15
Data Engineer                      10
Scrum Master                        3
Software Engineer (iOS)             3
DevOps Engineer                     3
Digital Product Manager             2
Machine Learning Engineer           2
Product Design (UX Researcher)      1
Software Architect                  1
Name: Pekerjaan, dtype: int6

Terlihat bahwa pada feature `PernahBekerja` walaupun ada 2 unique value namun kedua value tersebut punya maksud yang sama sehingga hanya ada 1 unique value pada feature tersebut, oleh sebab itu feature `PernahBekerja` akan di drop
<br> Pada feature `StatusPernikahan` terdapat value "-", value tersebut akan dimerge dengan value "Lainnya"

In [34]:
data_pre.drop(['PernahBekerja'], axis=1, inplace=True)

In [35]:
data_pre['StatusPernikahan'] = data_pre['StatusPernikahan'].replace({'-':'Lainnya'})

In [36]:
data_pre.shape

(285, 21)

In [37]:
data_pre.sample(5)

,StatusPernikahan,JenisKelamin,StatusKepegawaian,Pekerjaan,JenjangKarir,PerformancePegawai,AsalDaerah,HiringPlatform,SkorSurveyEngagement,SkorKepuasanPegawai,JumlahKeikutsertaanProjek,JumlahKeterlambatanSebulanTerakhir,JumlahKetidakhadiran,TingkatPendidikan,AlasanResign,TanggalLahir,TanggalHiring,TanggalResign,resign,umur,LamaBekerja
257,Menikah,Wanita,FullTime,Digital Product Manager,Freshgraduate_program,Sangat_kurang,Jakarta Timur,LinkedIn,4,4.0,0.0,0.0,15.0,Sarjana,masih_bekerja,1976-12-26,2013-07-08,2021-01-01,0,44.0,7.0
120,Bercerai,Pria,FullTime,Product Design (UI & UX),Freshgraduate_program,Kurang,Jakarta Barat,Indeed,4,3.0,6.0,0.0,17.0,Magister,masih_bekerja,1970-04-25,2017-02-10,2021-01-01,0,51.0,4.0
67,Bercerai,Pria,Outsource,Software Engineer (Back End),Mid_level,Kurang,Jakarta Barat,Employee_Referral,3,3.0,0.0,0.0,0.0,Doktor,masih_bekerja,1985-09-15,2015-03-30,2021-01-01,0,35.0,6.0
123,Lainnya,Wanita,Outsource,Software Engineer (Back End),Freshgraduate_program,Sangat_kurang,Jakarta Utara,Google_Search,3,5.0,0.0,0.0,5.0,Magister,masih_bekerja,1967-01-16,2011-01-10,2021-01-01,0,54.0,10.0
2,Menikah,Pria,FullTime,Software Engineer (Front End),Freshgraduate_program,Bagus,Jakarta Timur,Indeed,4,3.0,0.0,0.0,11.0,Magister,jam_kerja,1974-01-07,2011-01-10,2014-09-24,1,47.0,4.0


## Annual Report on Employee Number Changes

In [38]:
data_an = data_pre.copy()

In [39]:
data_an['tahun_hiring'] = pd.DatetimeIndex(data_an['TanggalHiring']).year
data_an['tahun_resign'] = pd.DatetimeIndex(data_an['TanggalResign']).year

In [40]:
#tabel agregat karyawan yang masuk
hire_agg = data_an.groupby(['tahun_hiring']).size().reset_index(name='jumlah_karyawan')
hire_agg

,tahun_hiring,jumlah_karyawan
0,2006,1
1,2007,2
2,2008,2
3,2009,7
4,2010,8
5,2011,76
6,2012,41
7,2013,43
8,2014,56
9,2015,30


In [41]:
#tabel agregat karyawan yang keluar
resign_agg = data_an.groupby(['tahun_resign']).size().reset_index(name='jumlah_karyawan')
resign_agg

,tahun_resign,jumlah_karyawan
0,2013,4
1,2014,11
2,2015,8
3,2016,8
4,2017,19
5,2018,26
6,2019,5
7,2020,6
8,2021,198


In [56]:
# outer join the two table based on year_hiring and year_resign
data_hiring_resign = pd.merge(hire_agg, resign_agg, left_on='tahun_hiring', right_on='tahun_resign', how='outer')
# use fillna() function to fill missing values with 0
data_hiring_resign.fillna(0, inplace=True)
# combine tahun_hiring and tahun_resign into one column 'tahun'
data_hiring_resign['tahun'] = data_hiring_resign[['tahun_hiring', 'tahun_resign']].max(axis=1)
# drop tahun_hiring and tahun_resign
data_hiring_resign.drop(['tahun_hiring','tahun_resign'], axis=1, inplace=True)
data_hiring_resign = data_hiring_resign[data_hiring_resign['tahun'] != 2021]
data_hiring_resign.rename(columns={'jumlah_karyawan_x': 'jumlah_hire', 'jumlah_karyawan_y': 'jumlah_resign'}, inplace=True)

In [57]:
data_hiring_resign

,jumlah_hire,jumlah_resign,tahun
0,1.0,0.0,2006.0
1,2.0,0.0,2007.0
2,2.0,0.0,2008.0
3,7.0,0.0,2009.0
4,8.0,0.0,2010.0
5,76.0,0.0,2011.0
6,41.0,0.0,2012.0
7,43.0,4.0,2013.0
8,56.0,11.0,2014.0
9,30.0,8.0,2015.0


In [69]:
min_tahun = data_hiring_resign['tahun'].min()
max_tahun = data_hiring_resign['tahun'].max()
fig = px.line(data_hiring_resign, x='tahun', y=['jumlah_hire','jumlah_resign'], title='Jumlah Karyawan Hiring dan Resign',
              template='plotly_dark',width=1000,height=800)
years = np.arange(min_tahun, max_tahun + 1)
fig.update_layout(xaxis={'tickvals':years, 'ticktext':years})
annotations = []
for i in range(1, len(data_hiring_resign)):
    jumlah_hire = data_hiring_resign.iloc[i]['jumlah_hire']
    jumlah_resign = data_hiring_resign.iloc[i]['jumlah_resign']
    tahun = data_hiring_resign.iloc[i]['tahun']
    jumlah_hire_sebelumnya = data_hiring_resign.iloc[i-1]['jumlah_hire']
    jumlah_resign_sebelumnya = data_hiring_resign.iloc[i-1]['jumlah_resign']
    if jumlah_hire != 0:
        persentase_kenaikan_hire = (jumlah_hire - jumlah_hire_sebelumnya) / jumlah_hire_sebelumnya * 100
        if persentase_kenaikan_hire >= 0:
            annotation_x = f'+{persentase_kenaikan_hire:.1f}%'
        else:
            annotation_x = f'{persentase_kenaikan_hire:.1f}%'
            annotations.append(dict(x=tahun, y=jumlah_hire, text=annotation_x, showarrow=True, arrowhead=2, ax=0, ay=-40))
    if jumlah_resign != 0:
        persentase_kenaikan_y = (jumlah_resign - jumlah_resign_sebelumnya) / jumlah_resign_sebelumnya * 100
        if persentase_kenaikan_y >= 0:
            annotation_y = f'+{persentase_kenaikan_y:.1f}%'
        else:
            annotation_y = f'{persentase_kenaikan_y:.1f}%'
        annotations.append(dict(x=tahun, y=jumlah_resign, text=annotation_y, showarrow=True, arrowhead=2, ax=0, ay=40))
for annotation in annotations:
    fig.add_annotation(annotation)
fig.show()